# Chp15. Big data and MapReduce

- 발표자 : 정지원
- 일자 : 2017.3.4(토)

---

__This chapter covers__
- MapReduce
- Using Python with Hadoop Streaming
- Automating MapReduce with mrjob
- Training support vector machines in parallel with the Pegasos algorithm

"네 예시가 좋지만, 내 데이터는 매우 큰데?" 라는 말을 종종 듣는다. 물론 이 책에서 사용하는 예제보다 당신이 다루는 데이터가 훨씬 크다는 것은 인정한다. 인터넷에 연결된 기기와 데이터기반의 의사결정에 흥미가 있는 사람들은 다루기 힘들 정도의 큰 사이즈의 데이터를 수집하곤 한다. 운 좋게도, 몇 가지 오픈 소스 소프트웨어 프로젝트가 이러한 큰 데이터를 다룰 수 있게 해준다. 먼저, _Hadoop_을 소개한다. 여러 기계에 데이터 처리를 분산시켜주는 Java Framework다.

너가 인터넷 쇼핑몰을 한다고 상상해봐라. 몇몇 사람들은 구매를 할테고, 몇몇은 구매하기 전에 떠날 것이다. 아마 구매자가 누군지 식별하길 원할 것이다. 어떻게 할까? 웹 서버 로그 기록을 보거나 각각의 사람들이 방문한 페이지를 봐야할 것이다. 만약에 다른 움직임들이 기록된다면 당신은 이러한 움직임(actions)을 분류기로 학습시킬 수 있을 것이다. 문제는 이 데이터셋이 매우 클 것이고, 하나의 기계로 학습시키기엔 매우 클 것이다. 이번 챕터는 이러한 문제들을 해결할 몇 가지 툴들을 보여준다. Hadoop과 Hodoop에 탑재된 몇몇 Python tools이다.

하둡은 MapReduce 프레임워크의 오픈소스 implementation이며 무료이다. 먼저 맵리듀스가 뭐고 하둡 프로젝트가 뭔지부터 알아야한다. 그리고 파이썬안에서 맵리듀스 작업을 어떻게 작성하는지 볼 것이다. 이러한 기능들을 하나의 기계에서 돌려보고, 나중에는 아마존 웹 서비스를 이용하여 Hadoop 작업들이 한 번에 여러 기계에서 어떻게 작동하는지 볼 것이다. 일단 맵리듀스 작업을 하는 것에 익숙해지면, 맵리듀스에서 머신러닝 작업을 하게 해주는 일반적인 솔루션에 대해 얘기해 볼 것이다. 그리고서는 _mrjob_이라 불리는 파이썬 안에서 맵리듀스 작업을 자동화 시키기 위한 프레임웍에 대해 볼 것이다. 마지막으로 여러 대의 기계에서 분류기를 학습시키는 데 사용할 수 있는 mrjob이 포함 된 분산 SVM을 작성해본다.

## 15.1 MapReduce: a framework for distributed computing

![Image](figures/1.png)

맵리듀스는 single 계산 작업을 여러대의 컴퓨터에 분산시켜주기 위한 소프트웨어 프레임웍이다. 한 대의 컴퓨터에서는 이러한 작업이 매우 오래 걸릴 것이라 생각을 하여, 시간을 단축하기 위해 여러대의 컴퓨터에서 작동을 시킬 것이다. 이러한 작업들 중 일부는 단일 시스템에서 실행하는 데 하루 이상 걸리는 일일 통계 요약이다.

미국 특허가 MapReduce를 Google에 발행했지만, 다른 회사들은 독자적으로 유사한 프레임 워크를 개발했다고 주장한다. 구글의 Jeffrey Dean과 Sanjay Ghemawat는 2004년에 처음으로 이 생각을 하였고, “MapReduce: Simplified Data Processing on Large Clusters.” 라는 논문을 냈다. 맵리듀스라는 이름은 _map_과 _reduce_에서 왔으며, 종종 함수형 프로그래밍이라 불리기도 한다.

맵리듀스는 클러스터에서 실행되며, 클러스터는 노드들로 구성되어 있다. 맵리듀스는 다음과 같이 작동한다. 하나의 작업(single job)이 작은 섹션들로 나뉘며, 입력 값은 각각 잘려서 노드들에게 분배된다. 각 노드들은 자신이 받은 데이터들만 처리한다. 각 노드에서 작동하는 코드들을 _mapper_라 하며, _map_ 과정으로 알려져있다. 개별 mapper로부터 나온 ouput은 몇몇 방법(주로 정렬)으로 combine된다. 정렬된 데이터는 더 작은 portions으로 쪼개지며 다음 처리를 하기 위해 노드로 분배된다. 이러한 두 번째 처리 과정을 _reduce_ 과정이라 하며, 코드 실행은 _reducer_로 알려져있다. reducer의 output은 당신이 찾는 최종 질문일 것이다.

MapReduce의 장점은 프로그램이 병렬적으로 실행될 수 있게 해준다는 것이다. 만약 클러스터가 10개의 노드를 가졌으며, 원래 작업이 10시간이 걸렸다면 MapReduce를 사용한다면 같은 결과를 1시간보다 아주 조금 더 걸린 시간안에 해결할 수 있을 것이다. 예를 들어, 최근 100년동안 중국에서의 최고 기온이 얼마인지 알고싶다고 해보자. 이 기간 동안 각 자역마다 유효한 데이터가 매일 있다고 가정한다. <지역> <일자> <온도>와 같은 데이터다. 우리는 가지고있는 노드의 수만큼 데이터를 분해 할 수 있으며 각 노드는 데이터 중에서 최대 값을 찾을 수 있다. 각 mapper는 하나의 온도, <"최대"> <온도>를 방출한다. 모든 mapper는 동일한 키를 생성한다. 이 키는 "최대"문자열이다. mapper의 출력을 비교하고 글로벌 최대 온도를 얻으려면 reducer가 하나만 있으면 된다.

__NOTE :__ 어느 시점에서든 개별 맵퍼 또는 리듀서는 서로 통신하지 않는다. 각 노드는 자체 비즈니스를 염두에 두고 할당 된 데이터를 계산한다.

작업 유형에 따라 다른 수의 reducer가 필요할 수 있다. 중국의 온도 예제를 다시 검토하고 매년 최고 기온을 원한다면 mapper는 매년 최대 기온을 찾아 배출해야하므로 중간 데이터는 <연도> <온도>와 같다. 이제 우리는 같은 해의 모든 값이 같은 reducer로가는 지 확인해야한다. 이는 map과 reduce 단계 사이의 정렬에서 수행된다. 이 예는 또 다른 요점인 데이터가 전달되는 방식을 보여준다. 데이터는 키/값 쌍으로 전달된다. 두 번째 중국의 온도 예제에서, 그 해는 key이었고 온도는 value였다. 우리는 연도별로 분류 했으므로 비슷한 해를 정확하게 결합했다. 각 reducer는 공통 키 (연도) 값을 수신한다.

이 예제로부터 알 수 있는 것은 reducer의 수는 고정이 아니라는 것이다. MapReduce는 많은 flexible한 옵션이 있다. 이러한 여러 조화는 master code로부터 다뤄진다. master node가 전체의 MapReduce job을 컨트롤하며, 어떤 데이터가 어느 노드에 존재할지 정해주고, mapping하고 sorting하고 reducing을 할 타이밍을 조절한다. master node는 또한 결함을 허용해준다. 종종 오류가 발생하면 mapper 입력 데이터의 여러 복사본이 여러 노드로 전송된다. 다음 그림을 통해 MapReduce 클러스터의 도식적 표현을 보자.

![Image](figures/2.png)

위 그림의 각 시스템에는 두 개의 프로세서가 있으며 두 개의 Map 또는 Reduce 작업을 동시에 처리 할 수 있다. Map 단계 중에 로봇 비행기에 의해 Machine 0이 파괴되면 마스터 노드는 이것을 인식한다. 마스터 노드가 장애를 감지하면 클러스터에서 Machine 0을 제거하고 작업을 계속한다. MapReduce의 일부 구현에서는 추가 데이터가 각 시스템에 저장된다. 기계 0에 대한 입력 데이터는 기계 0이 로봇 비행기에 의해 파괴되는 경우 기계 1에 저장 될 수도 있다. 일부 MapReduce 구현에서, 노드는 마스터 노드와 통신하여 그것이 살아 있고 작동 중임을 나타낸다. 그것들이 살아 있지 않고 정상적으로 기능하고 있다면, 마스터 노드는 마스터 노드를 재시작하거나 사용 가능한 머신 pool에서 완전히 제거 할 수 있다.

MapReduce에 대한 몇 가지 중요점은 다음과 같다.

- A master node controls the MapReduce job.
- MapReduce jobs are broken into map and reduce tasks.
- Map tasks don’t communicate with other map tasks; the same thing is true for reduce tasks.
- Between the map and reduce steps, there’s a sort or combine step.
- Redundant copies of the data are stored on separate machines in case of machine failure.
- Data is passed between mappers and reducers in key/value pairs.

## 15.2 Hadoop Streaming

하둡은 Java로 만들어진 오픈소스 프로젝트다. 하둡 프로젝트는 맵리듀스 작업을 수행하는데 가장 큰 특징을 가지고 있다. 분산 컴퓨팅에 더하여, 하둡은 분산 파일시스템을 가지고 있다.(distributed filesystem)

이 책은 Java 책도 아니고, Hadoop책도 아니다. 파이썬 상에서 Hadoop의 MapReduce가 어떤 식으로 작동하는지 정도만 알면 된다. 관심이 있다면, _Hadoop in Action_ 책을 보거나, http://hadoop.apache.org 를 참조해라.

하둡은 자바가 아닌 언어로 씌여진 분산 프로그램을 실행시킬 수 있게 해준다. 그러므로 파이썬에서도 맵리듀스 작업을 작성하여 실행시킬 수 있다. 하둡 스트리밍은 리눅스 환경에서 pipes와 같이 작동한다. | 기호를 사용하여 input을 output에 전달해주는 과정이다. 다음을 보자.

In [1]:
!ls -A

(MLA) Chp15 _ Big data and MapReduce.ipynb
.DS_Store
.ipynb_checkpoints
figures
inputFile.txt
kickStart.txt
mrMean.py
mrMeanMapper.py
mrMeanReducer.py
mrSVM.py
mrSVMkickStart.py
myLog.txt
mycred.txt
myout.txt
outFile.txt
outputFile.txt
pegasos.py
proximalSVM.py
svmDat2.txt
svmDat26
svmDat27
svmData.txt
testSet.txt
testSet200.txt


In [2]:
!cat inputFile.txt | python mapper.py | sort | python reducer.py > outputFile.txt

python: can't open file 'reducer.py': [Errno 2] No such file or directory
python: can't open file 'mapper.py': [Errno 2] No such file or directory


### 15.2.1 Distributed mean and variance mapper

숫자들의 평균과 분산을 구해주는 맵리듀스를 만들어보자. mrMeanMapper.py 코드는 다음과 같다.

In [3]:
import sys
from numpy import mat, mean, power
def read_input(file):
    for line in file:
        yield line.rstrip()
input = read_input(sys.stdin)
input = [float(line) for line in input]
numInputs = len(input)
input = mat(input)
sqInput = power(input,2)
print "%d\t%f\t%f" % (numInputs, mean(input), mean(sqInput))
print >> sys.stderr, "report: still alive"

0	nan	nan


/Users/jiwonjung/anaconda/envs/python2.7/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/Users/jiwonjung/anaconda/envs/python2.7/lib/python2.7/site-packages/numpy/core/_methods.py:68: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
report: still alive


직관적인 예제다. 모든 input lines을 돌고서 floats을 담은 list를 만든다. Numpy matrix를 만들어준다. 그리고 모든 값들의 제곱을 빠르게 구한다. 마지막으로 평균과 제곱의 평균들을 보내고, global mean과 variance를 구하는데 쓰이게 된다.

실제 실행시켜보자.

In [4]:
!source activate python2.7
!cat inputFile.txt | python2 mrMeanMapper.py

100	0.509570	0.344439
report: still alive


In [5]:
!python2 mrMeanMapper.py < inputFile.txt

100	0.509570	0.344439
report: still alive


첫 줄은 standard output이며, reducer에 보내게 된다. 두 번째 줄은 standard error의 상태이며, 마스터노드에 아직 잘 살아있음을 알리기 위해 보내진다.

### 15.2.2 Distributed mean and variance reducer

Mapper가 작동 했으므로 Reducer를 사용해 보자. Mapper는 raw number를 가져 와서 reducer의 중간 값으로 수집했다. 이러한 mappers를 병렬로 처리하고 모든 출력을 하나의 값으로 결합해야 한다. 이제 중간 키-값 쌍을 결합 할 수 있도록 Reducer를 작성해야한다.

In [6]:
import sys
from numpy import mat, mean, power
def read_input(file):
    for line in file:
        yield line.rstrip()
input = read_input(sys.stdin)
mapperOut = [line.split('\t') for line in input]
cumVal=0.0
cumSumSq=0.0
cumN=0.0
for instance in mapperOut:
    nj = float(instance[0])
    cumN += nj
    cumVal += nj*float(instance[1])
    cumSumSq += nj*float(instance[2])
mean = cumVal/cumN
varSum = (cumSumSq - 2*mean*cumVal + cumN*mean*mean)/cumN
print "%d\t%f\t%f" % (cumN, mean, varSum)
print >> sys.stderr, "report: still alive"

ZeroDivisionError: float division by zero

In [7]:
!cat inputFile.txt | python2 mrMeanMapper.py | python2 mrMeanReducer.py

report: still alive
100	0.509570	0.344439
report: still alive


In [8]:
!python2 mrMeanMapper.py < inputFile.txt | python2 mrMeanReducer.py

report: still alive
100	0.509570	0.344439
report: still alive


코드를 보면 Mapper로부터 output values를 받아서 사용한다. 어떻게 여러 대의 기계가 한 번에 작동하는지를 보았다. 아마 집에 10개의 서버를 두진 못할 것이다. 그러면 어떻게 몇 시간만에 서버들을 빠르게 빌리는지 알아보자.

## 15.3 Running Hadoop jobs on Amazon Web Services


동시에 100 대의 시스템에서 MapReduce 작업을 실행하려면 100 대의 시스템을 찾아야한다. 직접 구입할 수도 있고 아니면 다른 사람에게서 빌릴 수도 있다. Amazon은 http://aws.amazon.com/ 의 Amazon Web Services (AWS)를 통해 대규모 컴퓨팅 인프라의 일부를 개발자에게 대여해준다.

AWS는 웹 사이트, 스트리밍 비디오, 모바일 애플리케이션 등을 지원한다. 스토리지, 대역폭 및 컴퓨팅 성능이 측정되며 장기 계약이 없는 시간까지 이러한 각각의 용도로만 청구된다. 이것이 AWS가 매력적인 이유입니다. 사용하는 것에 대해서만 비용을 지불해라. 하루에 1,000 대의 컴퓨터를 사용하는 아이디어가 있다고 가정 해보자. AWS에서 설정하고 며칠 동안 실험을 할 수 있다. 그런 다음 나쁜 생각이라고 판단하면 시스템을 종료 할 수 있으며 더 이상 1,000 대의 컴퓨터에 비용을 지불 할 필요가 없다. 다음은 AWS에서 현재 사용할 수 있는 몇 가지 서비스에 대해 설명하고 AWS에서 설정하는 과정을 거쳐 AWS에서 Hadoop Streaming 작업을 실행한다.

### 15.3.1 Services available on AWS

AWS에서는 많은 수의 서비스를 사용할 수 있다. 그것들은 똘똘한 사람들에게 완전히 논리적인 것처럼 보이고 무식한 사람에게는 완전히 비밀스런것처럼 보이는 이름을 가지고 있다. AWS는 끊임없이 진화하고 새로운 서비스를 추가하고 있다. 네가 사용할 기본적인, 안정적인 서비스는 다음과 같다.

- _S3_ — Simple Storage Service is used for storing data on the internet and is used in conjunction with other AWS products. Here you’re renting some sort of stor- age device, so you pay by the amount of data you store and the length of time you store that data.

- _EC2_ — Elastic Compute Cloud is a service for using server instances. This is the heart of many common AWS-based systems. You can configure the server to run almost any operating system. A server can be started from a machine image in a matter of minutes. You can create, store, and share machine images. The elastic part of the name comes from the ability to quickly expand the number of serv- ers you’re running as your demands require.

- _Elastic MapReduce_ — This is the AWS MapReduce implementation. This is built on a slightly older release of Hadoop. (Amazon wanted to have a stable release, and so they made a few modifications, which prevents them from having the bleeding-edge release of Hadoop.) It has a nice GUI and simplifies the setup of Hadoop jobs. You don’t have to mess with adding files to Hadoop’s filesystem or configuring Hadoop machines. In Elastic MapReduce (EMR) you can run Java jobs or Hadoop Streaming jobs. We’ll focus on Hadoop Streaming.

많은 다른 서비스들이 존재한다. 비록 __S3__이 필요하지만 __EMR__에 초점을 맞출 것이다. EMR은 S3에서 파일을 가져 와서 Hadoop이 설치된 EC2 인스턴스를 시작한다.

### 15.3.2 Getting started with Amazon Web Services

AWS를 시작하기 위해선 계정이 필요하며, AWS를 사용하려면 신용카드를 등록해야 한다. 예시를 따라하기 위해서 약 1달러의 비용이 쓰인다. S3, EMR, EC2에 가입하면 된다.

## 15.4 Machine learning in MapReduce

10대의 컴퓨터에서 MapReduce를 사용하는 것은 현재 컴퓨터보다 10배나 좋은 컴퓨터를 사용하는 것과 다르다. MapReduce 작업이 올바르게 작성되면 이와 같이 느낄 수 있다. 그러나 모든 프로그램을 수행하고 즉각적인 속도 향상을 얻을 수는 없다. 그러려면 MapReduce 작업을 올바르게 작성되어야 한다.
많은 기계 학습 알고리즘은 MapReduce 프레임 워크에 직관적으로 적합하지 않다. 그건 중요하지 않다. 창의적인 과학자와 엔지니어는 거의 모든 대중적인 기계 학습 알고리즘에 MapReduce 솔루션을 작성했다. 다음은 이 책과 해당 MapReduce 구현에서 널리 사용되는 기계 학습 알고리즘을 간략하게 나열한 목록이다.

- Naïve Bayes—This is one of a few algorithms that’s naturally implementable in MapReduce. In MapReduce, it’s easy to calculate sums. In naïve Bayes, we were calculating the probability of a feature given a class. We can give the results from a given class to an individual mapper. We can then use the reducer to sum up the results.
- k-Nearest Neighbors—Trying to find similar vectors in a small dataset can take a large amount of time. In a massive dataset, it can limit daily business cycles. One approach to speed this up is to build a tree, such as a tree to narrow the search for closest vectors. This works well when the number of features is under 10. A pop- ular method for performing a nearest neighbor search on higher-dimensional items such as text, images, and video is locality-sensitive hashing.
- Support vector machines—The Platt SMO algorithm that we used in chapter 6 may be difficult to implement in a MapReduce framework. There are other implementations of SVMs that use a version of stochastic gradient descent such as the Pegasos algorithm. There’s also an approximate version of SVM called proximal SVM, which computes a solution much faster and is easily applied to a MapReduce framework.
- Singular value decomposition—The Lanczos algorithm is an efficient method for approximating eigenvalues. This algorithm can be applied in a series of MapReduce jobs to efficiently find the singular values in a large matrix. The Lanczos algorithm can similarly be used for principal component analysis.
- k-means clustering—One popular version of distributed clustering is known as canopy clustering. You can calculate the k-means clusters by using canopy clus- tering first and using the canopies as the k initial clusters.

MapReduce에서 몇 가지 일반적인 기계 학습 작업을 해결하는 방법을 배우고 싶다면 Apache Mahout 프로젝트 (http://mahout.apache.org/)와 Mahout in Action을 확인하면 된다. 
이 책은 아주 큰 데이터셋을 다루기 위한 일반적인 구현 세부 사항을 설명하는 데 특히 유용하다. Apache Mahout 프로젝트는 자바로 쓰여져 있다. 다른 맵리듀스에 대한 좋은 책으로는 Data Intensive Text Processing with Map/Reduce 가 있다.

이제 MapReduce 일을 처리하는 파이썬 툴에 대해 알아보자.

## 15.5 Using mrjob to automate Mapreduce in Python

이전에 소개된 알고리즘들 중 몇몇은 반복적이다. 그것들은 아주 작은 몇 개의 MapReduce로 완성될 수 있다. 만약 아주 큰 데이터셋에서 AdaBoost를 실행시키고 싶다면 어떠한가? 혹은 10개의 MapReduce 작업을 하고 싶다면 어떡할 것인가?

Cascading 및 Oozie와 같은 MapReduce 작업 흐름을 자동화하기위한 프레임 워크가 있지만 Amazon의 EMR에서 실행되는 것은 없다. Amazon의 EMR은 Python 스크립트를 사용할 수있는 Pig를 지원하지만이 작업을 수행하려면 다른 스크립팅 언어를 배워야한다. (Pig는 데이터 처리를 위해 더 높은 수준의 언어를 제공하는 Apache 프로젝트다. Pig는 데이터 처리 명령을 Hadoop MapReduce 작업으로 바꿔준다.) Python에서 MapReduce 작업을 실행하기 위한 몇 가지 도구가 있는데 그 중 저자가 좋아하는 것이 mrjob다.

Mrjob(http://packages.python.org/mrjob/)는 Yelp(식당 리뷰 웹 사이트)의 내부 프레임 워크로서 2010 년 말 오픈 소스를 만들었다. 이제 mrjob을 사용하는 방법을 알아본다. 그리고 나서 mrjob에서 이전에 했던 전역 평균과 분산 계산을 다시 작성해보겠다. mrjob을 유용하고 편리하게 사용할 수 있을 것이다. (Mrjob은 학습 도구로 유용하지만 여전히 Python이기 때문에 최상의 성능을 얻으려면 Java를 사용해야한다.)

### 15.5.1 Using mrjob for seamless intergration with EMR

Mrjob은 15.3 절에서와 같이 Elastic MapReduce에서 Hadoop Streaming을 실행한다. 가장 큰 차이점은 S3에 데이터를 업로드하고 명령을 올바르게 입력하는 것에 대해 걱정할 필요가 없다는 것이다. 이 모든 것은 mrjob에 의해 처리됩니다. mrjob을 사용하면 자신의 Hadoop 클러스터나 비분산(nondistributed) 모드에서 MapReduce 작업을 테스트 할 수 있습니다. 작업을 로컬에서 실행하여 EMR에서 실행하는 작업으로의 전환이 쉽다.

예를 들어, 로컬로 작업을 실행하기 위해, 다음과 같이 치면 된다.

In [9]:
!python2 mrMean.py < inputFile.txt > myOut.txt

No configs found; falling back on auto-configuration
Creating temp directory /var/folders/jw/f2v6jst509n730w8mh0x87sc0000gn/T/mrMean.jiwonjung.20170303.172637.205424
mr() is deprecated and will be removed in v0.6.0. Use mrjob.step.MRStep directly instead.
Running step 1 of 1...
reading from STDIN
mr() is deprecated and will be removed in v0.6.0. Use mrjob.step.MRStep directly instead.
mr() is deprecated and will be removed in v0.6.0. Use mrjob.step.MRStep directly instead.
mr() is deprecated and will be removed in v0.6.0. Use mrjob.step.MRStep directly instead.
mr() is deprecated and will be removed in v0.6.0. Use mrjob.step.MRStep directly instead.
mr() is deprecated and will be removed in v0.6.0. Use mrjob.step.MRStep directly instead.
mr() is deprecated and will be removed in v0.6.0. Use mrjob.step.MRStep directly instead.
Streaming final output from /var/folders/jw/f2v6jst509n730w8mh0x87sc0000gn/T/mrMean.jiwonjung.20170303.172637.205424/output...
Removing temp directory /var/folder

EMR에서 동일한 작업을 실행하려면 다음과 같이 입력하면 된다.

In [10]:
!python2 mrMean.py -r emr < inputFile.txt > myOut.txt

No configs found; falling back on auto-configuration
Traceback (most recent call last):
  File "mrMean.py", line 48, in <module>
    MRmean.run()
  File "/usr/local/lib/python2.7/site-packages/mrjob/job.py", line 439, in run
    mr_job.execute()
  File "/usr/local/lib/python2.7/site-packages/mrjob/job.py", line 460, in execute
    super(MRJob, self).execute()
  File "/usr/local/lib/python2.7/site-packages/mrjob/launch.py", line 161, in execute
    self.run_job()
  File "/usr/local/lib/python2.7/site-packages/mrjob/launch.py", line 229, in run_job
    with self.make_runner() as runner:
  File "/usr/local/lib/python2.7/site-packages/mrjob/job.py", line 485, in make_runner
    return super(MRJob, self).make_runner()
  File "/usr/local/lib/python2.7/site-packages/mrjob/launch.py", line 172, in make_runner
    return EMRJobRunner(**self.emr_job_runner_kwargs())
  File "/usr/local/lib/python2.7/site-packages/mrjob/emr.py", line 631, in __init__
    self._fix_s3_tmp_and_log_uri_opts()
  File 

우리가 15.3 절에서했던 모든 업로드 및 양식 작성은 mrjob에서 자동으로 수행된다. 로컬 Hadoop 클러스터에 작업을 실행하기 위해 다른 명령을 추가 할 수 있다. 또한 수 많은 Command-line 인자를 추가하여 EMR 또는 서버 유형에서 원하는 서버 수를 지정할 수 있다. 15.3절에서 Mapper와 Reducer를 위한 두 개의 개별 파일이 있었다. mrjob에서 mapper와 reducer는 같은 스크립트에 있을 수 있다. 이제 스크립트 내부를 살펴보고 어떻게 작동하는지 알아 보자.

### 15.5.2 The anatomy of a MapReduce script in mrjob

mrjob으로 많은 일을 할 수 있지만, 시작하기 위해 전형적인 Map 작업을 살펴본다. 우리는 프레임 워크의 미묘함에 집중할 수 있도록 동일한 평균과 분산 문제를 해결할 것이다.

In [11]:
from mrjob.job import MRJob
class MRmean(MRJob):
    def __init__(self, *args, **kwargs):
        super(MRmean, self).__init__(*args, **kwargs)
        self.inCount = 0
        self.inSum = 0
        self.inSqSum = 0
    def map(self, key, val):
        if False: yield
        inVal = float(val)
        self.inCount += 1
        self.inSum += inVal
        self.inSqSum += inVal*inVal
    def map_final(self):
        mn = self.inSum/self.inCount
        mnSq = self.inSqSum/self.inCount
        yield (1, [self.inCount, mn, mnSq])
    def reduce(self, key, packedValues):
        cumVal=0.0; cumSumSq=0.0; cumN=0.0
        for valArr in packedValues:
            nj = float(valArr[0])
            cumN += nj
            cumVal += nj*float(valArr[1])
            cumSumSq += nj*float(valArr[2])
        mean = cumVal/cumN
        var = (cumSumSq - 2*mean*cumVal + cumN*mean*mean)/cumN
        yield (mean, var)

def steps(self):
    return ([self.mr(mapper=self.map, reducer=self.reduce, mapper_final=self.map_final)])

if __name__ == '__main__':
    MRmean.run()

Usage: __main__.py [options] [input files]

__main__.py: error: no such option: -f


SystemExit: 2

/Users/jiwonjung/anaconda/envs/python2.7/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


이 코드는 분산 평균 및 분산을 계산한다. 입력 텍스트는 여러 Mapper로 나뉘며 중간 값을 계산하여 Reducer에 누적되어 전역 평균 및 분산을 제공한다.

MRjob 클래스에서 상속받은 새 클래스를 만들어야 한다. 이 예에서는 클래스 MRmean을 호출했다. Mapper와 Reducer는 이 클래스의 메소드다. steps()라고하는 또 다른 메소드가 있는데, 취해진 단계를 정의한다. map-reduce를 할 필요는 없다. map-reduce-reduce-reduce 또는 map-reduce-map-reduce-map-reduce를 할 수 있다. 이에 대한 예제는 다음 섹션에서 볼 수 있다. steps () 메서드에서 mrjob에 Mapper와 Reducer의 이름을 알려준다. 아무 것도 지정하지 않으면 Mapper 및 Reducer라는 메서드가 검색된다.

Mapper의 동작에 대해 이야기 해보자. mapper는 for 루프의 내부와 같이 작동하며 모든 입력 행을 호출한다. 입력의 모든 줄을받은 후에 무언가를 하고 싶다면 mapper_final에서 할 수 있다. 처음 엔 이상하게 보일 수도 있지만 실제로는 편리하다. mapper()와 mapper_final() 사이에서 상태를 공유 할 수 있다. 그래서 예제에서 mapper()에 입력 값을 누적하고, 모든 값을 가질 때 우리는 제곱 값의 평균과 평균을 계산하여 보내준다. 값은 yield 문을 통해 매퍼에서 전송된다.

값은 키/값 쌍으로 표시된다. 여러 값을 보내려면 목록에 값을 넣는 것이 좋다. 값은 맵 단계 다음에 키순으로 정렬된다. Hadoop은 일을 정렬하는 방법을 변경할 수 있는 옵션을 가지고 있지만 대부분의 어플리케이션에서 기본 정렬이 작동해야한다. 동일한 키 값을 갖는 값은 동일한 Reducer로 전송된다. 정렬 단계 후에 유사한 값이 함께 수집되도록 키에 사용하는 것을 생각해야 한다. 하나의 Reducer가 필요하기 때문에 모든 Mapper 출력에 대해 키 1을 사용했으며 모든 Mapper 출력을 동일한 Reducer에서 감당하길 원한다.

mrjob의 Reducer는 Mapper와 다르게 동작한다. Reducer에서 입력은 반복 가능한 객체로 표시된다. 이것들을 반복하려면, for 루프 같은 것을 사용할 필요가 있다. mapper 또는 mapper_final과 Reducer 사이에서 상태를 공유 할 수 없습니다. 그 이유는 파이썬 스크립트가 맵에서 살아남지 못하고 단계를 줄이기 때문이다. mapper와 reducer간에 통신하려면 키/값 쌍을 통해 수행해야 한다. Reducer의 맨 아래에 출력이 예정되어 있기 때문에 키가 없는 출력을 추가했다. 그들이 다른 mapper에게 간다면, 나는 key value를 넣을 것이다.

이제 충분하다. 실제로 해보자. Mapper 만 실행하려면 파이썬 쉘이 아닌 Linux / DOS 창에 다음 명령을 입력하십시오. inputFile.txt 파일은 Ch15 코드 다운로드에 있다.

In [12]:
!python2 mrMean.py --mapper < inputFile.txt

No handlers could be found for logger "mrjob.job"
1	[100, 0.5095697, 0.34443931307936]


full function을 실행하려면 --mapper 옵션을 지워봐라.

In [13]:
!python2 mrMean.py < inputFile.txt

No configs found; falling back on auto-configuration
Creating temp directory /var/folders/jw/f2v6jst509n730w8mh0x87sc0000gn/T/mrMean.jiwonjung.20170303.172643.244766
mr() is deprecated and will be removed in v0.6.0. Use mrjob.step.MRStep directly instead.
Running step 1 of 1...
reading from STDIN
mr() is deprecated and will be removed in v0.6.0. Use mrjob.step.MRStep directly instead.
mr() is deprecated and will be removed in v0.6.0. Use mrjob.step.MRStep directly instead.
mr() is deprecated and will be removed in v0.6.0. Use mrjob.step.MRStep directly instead.
mr() is deprecated and will be removed in v0.6.0. Use mrjob.step.MRStep directly instead.
mr() is deprecated and will be removed in v0.6.0. Use mrjob.step.MRStep directly instead.
mr() is deprecated and will be removed in v0.6.0. Use mrjob.step.MRStep directly instead.
Streaming final output from /var/folders/jw/f2v6jst509n730w8mh0x87sc0000gn/T/mrMean.jiwonjung.20170303.172643.244766/output...
0.5095697	0.08477803392127012
Remov

마지막으로 Amazon Elastic MapReduce에서 실행하려면이 다음 명령을 입력해라. (AWS_ACCESS_KEY_ID 및 AWS_SECRET_ACCESS_KEY 환경 변수가 설정되어 있는지 확인해라. 설정 방법은 부록 A를 참조)

In [14]:
!python2 mrMean.py -r emr < inputFile.txt > outFile.txt

No configs found; falling back on auto-configuration
Traceback (most recent call last):
  File "mrMean.py", line 48, in <module>
    MRmean.run()
  File "/usr/local/lib/python2.7/site-packages/mrjob/job.py", line 439, in run
    mr_job.execute()
  File "/usr/local/lib/python2.7/site-packages/mrjob/job.py", line 460, in execute
    super(MRJob, self).execute()
  File "/usr/local/lib/python2.7/site-packages/mrjob/launch.py", line 161, in execute
    self.run_job()
  File "/usr/local/lib/python2.7/site-packages/mrjob/launch.py", line 229, in run_job
    with self.make_runner() as runner:
  File "/usr/local/lib/python2.7/site-packages/mrjob/job.py", line 485, in make_runner
    return super(MRJob, self).make_runner()
  File "/usr/local/lib/python2.7/site-packages/mrjob/launch.py", line 172, in make_runner
    return EMRJobRunner(**self.emr_job_runner_kwargs())
  File "/usr/local/lib/python2.7/site-packages/mrjob/emr.py", line 631, in __init__
    self._fix_s3_tmp_and_log_uri_opts()
  File 

## 15.6 Example: the Pegasos algorithm for distributed SVMs

4장에서 naïve Bayes라고하는 텍스트 분류 알고리즘을 봤었다. 4장에서 텍스트 문서를 Word 공간에서 벡터로 취급했습니다. 6장에서는 분류를 위해 서포트 벡터 머신 또는 SVM을 살펴 보았다. SVM은 텍스트 분류에서 잘 작동 할 수 있다. 우리는 각 문서를 수만 가지 기능을 가진 벡터로 취급한다.

많은 수의 문서에 텍스트 분류를 하는 것은 기계 학습에 있어 큰 도전이다. 많은 양의 데이터에 대해 Classifier를 훈련시키는 방법은 무엇인가? 알고리즘을 병렬 작업으로 분해 할 수 있다면 Map Re-ce 프레임 워크가 도움이 될 수 있다. 6장에서 기억한다면 SMO 알고리즘은 한 번에 두 개의 지원 벡터를 최적화했다. SMO 알고리즘은 전체 데이터 세트를 반복하면서 주의가 필요한 값으로 멈춘다. 이 알고리즘은 병렬화하기 쉽지 않다.

![Images](figures/3.png)

SMO 알고리즘의 한 가지 대안은 Pegasos 알고리즘이다. Pegasos 알고리즘은 MapReduce 용으로 쉽게 작성할 수 있다. 다음에 Pegasos 알고리즘을 알아본 후 Pegasos의 분산 버전을 작성하는 방법을 살펴 보겠다. 마지막으로 mrjob에서 분산 버전을 실행한다.

### 15.6.1 The Pegasos algorithm

Pegasos는 Primal Estimated sub-GrAdient Solver의 머리 글자다. 이 알고리즘은 지원 벡터 머신에 의해 정의 된 최적화 문제를 해결하기 위해 Stochastic Gradient Descent 방식을 사용한다. 필요한 반복 횟수는 데이터 집합의 크기가 아니라 원하는 정확도에 의해 결정된다.

6장에서 서포트 벡터 머신에서 분리 된 초평면을 찾으려고 한다는 것을 기억해봐라. 우리의 2차원적인 예제에서 우리는 두 클래스의 데이터를 적절하게 구분하는 선을 찾으려고 한다. Pegasos 알고리즘은 다음과 같이 작동한다. 우리의 훈련 데이터에서 무작위로 선택된 점들의 집합이 배치에 추가된다. 이러한 각 점은 제대로 분류되었는지 확인하기 위해 테스트된다. 잘 분류 됐다면 무시된다. 제대로 분류되지 않은 경우 업데이트 세트에 추가된다. 일괄 처리가 끝나면 가중치 벡터가 부적절하게 분류 된 벡터로 업데이트된다. 이렇게 반복된다.

슈도코드는 다음과 같다.


>*Set w to all zeros*

>*For each batch*
>>*Choose k data vectors randomly*

>>*For each vector*
>>>*If the vector is incorrectly classified:*
>>>>*Change the weights vector: w*

>*Accumulate the changes to w*

In [15]:
def predict(w, x):
    return w*x.T
def batchPegasos(dataSet, labels, lam, T, k):
    m,n = shape(dataSet); w = zeros(n);
    dataIndex = range(m)
    for t in range(1, T+1):
        wDelta = mat(zeros(n))
        eta = 1.0/(lam*t)
        random.shuffle(dataIndex)
        for j in range(k):
            i = dataIndex[j]
            p = predict(w, dataSet[i,:])
            if labels[i]*p < 1:
                wDelta += labels[i]*dataSet[i,:].A
        w = (1.0 - 1/t)*w + (eta/k)*wDelta
    return w

Listing 15.4의 코드는 Pegasos 알고리즘의 순차적 버전이다. 입력 T와 k는 각각 반복 횟수와 배치 크기를 설정한다. 각 반복에서 학습 속도 또는 가중치의 변경 정도를 결정하는 η를 다시 계산한다. 외부 루프에서는 다음 배치에서 사용할 새 데이터 세트를 선택한다. 안쪽 루프는 일괄 처리다. 여기서 B는 잘못 분류 된 값의 값을 누적 한 다음 가중치 벡터를 업데이트한다.

이 예제를 사용하려면 6장의 일부 데이터를 사용하여 이 예제를 실행할 수 있다. 우리는 이 코드를 MapReduce 버전의 시작점으로 사용하는 것을 제외하고는 많은 것을 하지 않는다. 다음 섹션에서는 mrjob에 Pegasos의 MapReduce 버전을 빌드하고 실행할 것이다.

### 15.6.2 Training: MapReduce support vector machines with mrjob

우리는 MapReduce의 Listing 15.4에서 Pegasos 알고리즘을 구현할 것이다. 알고리즘을 구현하기 위해 15.5 절에서 살펴볼 mrjob 프레임 워크를 사용할 것이다. 먼저 알고리즘을 맵으로 분해하고 단계를 줄이는 방법을 결정해야한다. 무엇을 병행할 수 있는가? 병렬로 처리 할 수 없는 것은 무엇인가?

15.4에서 코드를 실행할 때 진행되는 모든 계산을 살펴 본다면 많은 시간이 내부 제품을 수행하는 데 소비된다는 것을 알 수 있다. 이들 내부 products는 병렬화 할 수 있지만 새로운 w 벡터의 생성을 병렬화 할 수는 없다. 이것은 MapReduce 작업을 작성하기위한 좋은 출발점을 제공한다. Mapper와 Reducer를 작성하기 전에 supporting 코드를 작성해 보겠다.

In [16]:
from mrjob.job import MRJob

import pickle
from numpy import *

class MRsvm(MRJob):
    DEFAULT_INPUT_PROTOCOL = 'json_value'
    def __init__(self, *args, **kwargs):
        super(MRsvm, self).__init__(*args, **kwargs)
        self.data = pickle.load(open(\
                 '<path to your Ch15 code directory>\svmDat27'))
        self.w = 0
        self.eta = 0.69
        self.dataList = []
        self.k = self.options.batchsize
        self.numMappers = 1
        self.t = 1
    def configure_options(self):
        super(MRsvm, self).configure_options()
        self.add_passthrough_option(
            '--iterations', dest='iterations', default=2, type='int',
            help='T: number of iterations to run')
        self.add_passthrough_option(
            '--batchsize', dest='batchsize', default=100, type='int',
            help='k: number of data points in a batch')
    def steps(self):
        return ([self.mr(mapper=self.map, mapper_final=self.map_fin,\
                reducer=self.reduce)]*self.options.iterations)

if __name__ == '__main__':
    MRsvm.run()

Usage: __main__.py [options] [input files]

__main__.py: error: no such option: -f


SystemExit: 2

Listing 15.5의 코드는 모든 것을 설정하므로 맵을 수행하고 단계를 적절하게 줄일 수 있다. MRsvm이라는 mrjob 클래스를 만만든다. **__init __** () 메소드는 맵에서 사용하고 단계를 줄이는 일부 변수를 초기화한다. Python 모듈 Pickle은 다른 버전의 Python으로 pickle한 파일을 Load하는 것을 싫어한다. 파이썬 2.6과 2.7로 pickled 된 데이터 파일을 각각 svmDat26과 svmDat27이라는 파일에 포함 시켰다.

configure_options () 메소드는 명령 행에서 입력 할 수 있는 값을 설정한다. 이것들은 반복 횟수 (T)와 일괄 처리 크기 (k)다. 이러한 인수는 모두 선택 사항이며, 설정되지 않은 경우 15.5의 값을 기본값으로 사용한다.

마지막으로 steps () 메서드는 mrjob에게 어떤 작업을 어떤 순서로 수행할지 알려준다. map, map_fin 및 reduce steps을 사용하여 파이썬 목록을 만든 다음 이를 반복 횟수만큼 곱하여 각 반복마다 목록을 반복한다. 이 작업 체인이 제대로 작동하려면 Mapper가 Reducer에서 나오는 데이터를 읽을 수 있어야한다. 우리는 단일 MapReduce 작업에서 이 요구 사항을 충족시키지 못했기 때문에 입력과 출력에 더욱 주의를 기울여야한다.

입력과 출력을 천천히 정의해보자.

__Mapper__

- Inputs: < mapperNum, valueList >
- Outputs: nothing

__Mapper_final__

- Inputs: nothing
- Outputs: < 1, valueList >

__Reducer__

- Inputs: < mapperNum, valueList >
- Outputs: < mapperNum, valueList >

전달 된 값은 lists다. list의 첫 번째 값은 나머지 list에 저장된 데이터 유형을 설명하는 문자열이다. valueList의 두 가지 예는 ['x', 23] 및 ['w', [1,5,6]]다. 모든 mapper_final은 동일한 키를 방출한다. 이것은 모든 키/값 쌍이 하나의 Reducer에 있는지 확인하는 것이다.

입력과 출력을 정의한 다음 Mapper와 Reducer 메소드를 작성해 보겠다.

In [17]:
def map(self, mapperId, inVals):
    if False: yield
    if inVals[0]=='w':
        self.w = inVals[1]
    elif inVals[0]=='x':
        self.dataList.append(inVals[1])
    elif inVals[0]=='t': self.t = inVals[1]
        
def map_fin(self):
    labels = self.data[:,-1]; X=self.data[:,0:-1]
    if self.w == 0: self.w = [0.001]*shape(X)[1]
    for index in self.dataList:
        p = mat(self.w)*X[index,:].T
        if labels[index]*p < 1.0:
            yield (1, ['u', index])
    yield (1, ['w', self.w])
    yield (1, ['t', self.t])

def reduce(self, _, packedVals):
    for valArr in packedVals:
        if valArr[0]=='u':  self.dataList.append(valArr[1])
        elif valArr[0]=='w': self.w = valArr[1]
        elif valArr[0]=='t':  self.t = valArr[1]
    labels = self.data[:,-1]; X=self.data[:,0:-1]
    wMat = mat(self.w);   wDelta = mat(zeros(len(self.w)))
    for index in self.dataList:
        wDelta += float(labels[index])*X[index,:]
    eta = 1.0/(2.0*self.t)
    wMat = (1.0 - 1.0/self.t)*wMat + (eta/self.k)*wDelta
    for mapperNum in range(1,self.numMappers+1):
        yield (mapperNum, ['w', wMat.tolist()[0] ])
        if self.t < self.options.iterations:
            yield (mapperNum, ['t', self.t+1])
            for j in range(self.k/self.numMappers):
                yield (mapperNum, ['x', random.randint(shape(self.data)[0]) ])

Listing 15.6의 첫 번째 메소드는 map()이다. 이것은 분배된 부분이다. 값을 받아 map_fin()에서 처리하기 위해 값을 저장한다. 입력은 w벡터, t값 또는 x의 세 가지 유형 중 하나 일 수 있다. t값은 반복 수이며 이 계산에 사용되지 않는다. 상태를 저장할 수 없기 때문에 한 반복에서 다음 반복까지 원하는 변수가 있으면 키/값 쌍으로 전달하거나 disk에 쓸 수 있습니다. 이 값을 전달하는 것이 더 쉽고 빠르다.

모든 입력이 도착하면 map_fin() 메서드가 실행된다. 이 시점에서 가중치 벡터 W와 이 일괄 처리를 위한 x값 목록을 갖게 될 것이다. 각 x값은 정수다. 그것은 데이터가 아니라 인덱스다. 데이터는 디스크에 저장되고 스크립트가 실행될 때 메모리에 로드된다. map_fin()이 시작되면 데이터를 레이블과 데이터로 분할한다. 그런 다음 self.dataList에 저장된 현재 일괄 처리의 모든 값을 반복한다. 이러한 값 중 하나라도 잘못 분류 된 경우 Reducer에 방출된다. Mapper와 Reducer 사이의 상태를 유지하기 위해 w 벡터와 t 값이 감속기로 전송된다.

마지막으로 단 하나의 Reducer만 있어야 한다. 먼저 모든 키/값 쌍을 반복하고 값을 지역 변수에 압축을 푼다. dataList의 값은 w벡터를 업데이트하는데 사용된다. 업데이트는 wDelta에 누적된다. 그러면 wMat가 wDelta와 학습률 eta에 의해 업데이트된다. wMat가 업데이트 된 후에는 전체 프로세스를 시작해야한다. 새로운 무작위 벡터 배치가 선택되어 방출된다. 이 모든 값의 핵심은 Mapper 번호다.

이를 실제로 보려면 Reducer에서 나온 것처럼 보이는 일부 데이터로 작업을 시작해야한다. 이 작업을 수행할 kickStart.txt라는 파일을 첨부했다. 로컬 시스템에서 이전 코드를 실행하려면 다음 명령을 입력해라.

In [18]:
!python2 mrSVM.py < kickStart.txt

No configs found; falling back on auto-configuration
Creating temp directory /var/folders/jw/f2v6jst509n730w8mh0x87sc0000gn/T/mrSVM.jiwonjung.20170303.172651.155329
mr() is deprecated and will be removed in v0.6.0. Use mrjob.step.MRStep directly instead.
Running step 1 of 2...
reading from STDIN
mr() is deprecated and will be removed in v0.6.0. Use mrjob.step.MRStep directly instead.
mr() is deprecated and will be removed in v0.6.0. Use mrjob.step.MRStep directly instead.
mr() is deprecated and will be removed in v0.6.0. Use mrjob.step.MRStep directly instead.
mr() is deprecated and will be removed in v0.6.0. Use mrjob.step.MRStep directly instead.
mr() is deprecated and will be removed in v0.6.0. Use mrjob.step.MRStep directly instead.
mr() is deprecated and will be removed in v0.6.0. Use mrjob.step.MRStep directly instead.
Running step 2 of 2...
mr() is deprecated and will be removed in v0.6.0. Use mrjob.step.MRStep directly instead.
mr() is deprecated and will be removed in v0.6.0. 

출력 벡터가 여기에 주어진다. 이 벡터의 플롯은 그림 15.9에 2회 반복과 50회 반복으로 나타납니다.

EMR에서 작업을 실행하려면 -r emr 명령을 추가해라. 이 작업에 사용할 서버의 기본 수는 1입니다. 이를 늘리려면 --num-ec2-instances = 2 또는 원하는 양의 정수를 추가해라. 다음은 그 예시다.

In [19]:
!python2 mrSVM.py -r emr --num-ec2-instances=3 < kickStart.txt > myLog.txt

No configs found; falling back on auto-configuration
num_ec2_instances is deprecated; set num_core_instances to 2 instead
Traceback (most recent call last):
  File "mrSVM.py", line 89, in <module>
    MRsvm.run()
  File "/usr/local/lib/python2.7/site-packages/mrjob/job.py", line 439, in run
    mr_job.execute()
  File "/usr/local/lib/python2.7/site-packages/mrjob/job.py", line 460, in execute
    super(MRJob, self).execute()
  File "/usr/local/lib/python2.7/site-packages/mrjob/launch.py", line 161, in execute
    self.run_job()
  File "/usr/local/lib/python2.7/site-packages/mrjob/launch.py", line 229, in run_job
    with self.make_runner() as runner:
  File "/usr/local/lib/python2.7/site-packages/mrjob/job.py", line 485, in make_runner
    return super(MRJob, self).make_runner()
  File "/usr/local/lib/python2.7/site-packages/mrjob/launch.py", line 172, in make_runner
    return EMRJobRunner(**self.emr_job_runner_kwargs())
  File "/usr/local/lib/python2.7/site-packages/mrjob/emr.py", li

To see all of the options available, enter %python mrSVM.py –h.

![Images](figures/4.png)
![Images](figures/5.png)


많은 machine에서 기계 학습 작업을 작성하고 시작하는 방법을 알았으므로 이제는 이 작업이 실제로 필요한지 여부에 대해 이야기 해보자.

## 15.7 Do you really need MapReduce?

나는 당신이 누구인지 몰라도 MapReduce나 Hadoop이 필요 없다고 말할 수 있다. 대다수의 컴퓨팅 요구가 단일 컴퓨터로 충족 될 수 있기 때문이다. 이 대용량 데이터 도구는 Google, Yelps 및 Facebook에 의해 만들어졌지만 그러한 회사가 몇이나 있는가?

Resources를 최대한 활용하면 시간과 에너지를 절약 할 수 있다. 컴퓨터 작업이 너무 오래 걸리면 다음 질문을 해봐라. C 또는 Java와 같은 보다 효율적인 언어로 코드를 재작성할 수 있는가? 이미 해당 언어 중 하나를 사용하고 있다면 가장 효율적으로 코드를 작성하고 있는가? 처리량이 메모리 또는 프로세서에 의해 제한되는가? 아마도 당신은 이 질문들에 대한 답을 모를 것이다.

대부분의 사람들은 한 대의 컴퓨터에서 얼마나 많은 작업을 처리 할 수 있는지 알지 못한다. Big data 문제가 없다면 MapReduce 및 Hadoop이 필요하지 않다. 당신이 big data 문제가 있는 경우에 뭘 해야할지 아는 것은 중요할 것이다.

## 15.8 Summary

컴퓨팅 요구가 컴퓨팅 리소스의 기능을 초과하면 더 나은 시스템을 구입하는 것이 좋다. 합리적으로 가격이 책정 된 기계의 능력을 당신의 컴퓨터가 필요 이상으로 초과했을 수도 있다. 한 가지 해결책은 컴퓨팅을 병렬 작업으로 분해하는 것이다. 이를 수행하는 한 가지 패러다임은 MapReduce다. MapReduce에서는 작업을 Mapping하고 단계를 줄인다.

일반적인 작업에서는 Map 단계를 사용하여 데이터를 병렬로 처리 한 다음 Reduce 단계에서 데이터를 결합 할 수 있다. 이 N:1 모델은 전형적이지만 작업을 결합하는 유일한 방법은 아니다. 데이터는 키/값 쌍을 사용하여 맵퍼와 리듀서간에 전달된다. 일반적으로 데이터는 맵 단계 이후의 키 값에 따라 정렬된다. Hadoop은 MapReduce 작업을 실행하기 위한 인기있는 Java 프로젝트다. Hadoop에는 Hadoop Streaming이라는 Java 이외의 작업을 실행하기 위한 응용 프로그램이 있다.

Amazon Web Services를 사용하면 시간당 컴퓨팅 리소스를 임대 할 수 있다. Amazon Web Services에서 제공하는 도구 중 하나는 Elastic MapReduce이며 사람들이 Hadoop Streaming 작업을 실행할 수 있도록한다. 간단한 1단계 MapReduce 작업은 Elastic MapReduce 관리 콘솔에서 작성하고 실행할 수 있다. 더 복잡한 작업에는 추가 도구가 필요하다. 상대적으로 새로운 오픈 소스 도구 중 하나가 mrjob이다. mrjob을 사용하면 여러 MapReduce 작업을 순차적으로 실행할 수 있다. 최소한의 설정으로 mrjob은 Amazon Web Services와 관련된 지저분한 단계를 처리한다.

MapReduce 작업으로 여러 가지 기계 학습 알고리즘을 쉽게 작성할 수 있다. 일부 기계 학습 작업은 MapReduce에서 사용하기 위해 창의적으로 재정의되어야한다. 서포트 벡터 머신은 텍스트 분류를 위한 강력한 도구이지만 많은 수의 문서에서 분류자를 훈련 시키려면 많은 양의 컴퓨팅 리소스가 필요하다. 벡터 머신 지원을 위한 분산 분류기를 생성하는 한 가지 방법은 Pegasos 알고리즘이다. Pegasos와 같은 여러 MapReduce 작업이 필요할 수 있는 기계 학습 알고리즘은 mrjob에서 쉽게 구현된다.

이것으로 이 책의 주요 내용이 마무리된다. 이 책이 당신에게 새로운 문을 열어주기를 바란다. 기계 학습의 수학이나 코드의 실제 구현에서 더 많은 것을 탐구해야한다. 이 책의 도구와 기술을 사용하여 흥미로운 응용 프로그램을 만들 것을 기대한다.